In [1]:
from pyqtgraph.Qt import QtGui, QtCore
import numpy as np
import pyqtgraph as pg
import h5py
import os
import fnmatch
import math

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
listOfFiles = os.listdir('./')
pattern = "*.mat"
for entry in listOfFiles:
    if fnmatch.fnmatch(entry, pattern):
        print(entry)

inv_opal_nz_10_r_100nm_th_5nm_TE.mat


In [3]:
d = h5py.File("inv_opal_nz_10_r_100nm_th_5nm_TE.mat","r")
print(list(d.keys()))

['E2', 'Ex', 'Ey', 'Ez', 'Ngen', 'Pabs', 'R_nopavg', 'T_nopavg', 'f', 'n', 'n_oxide', 'n_x', 'n_y', 'n_z', 'nf', 'wl', 'x', 'y', 'z']


In [4]:
x = np.array(d.get('x'))
y = np.array(d.get('y'))
z = np.array(d.get('z'))
f = np.array(d.get('f'))
nx = np.size(x)
ny = np.size(y)
nz = np.size(z)
nf = np.size(f)
wl = np.reshape(np.transpose(np.array(d.get('wl')))*1e3,(nf,))
R = np.reshape(np.transpose(np.array(d.get('R_nopavg'))),(nf,))
T = np.transpose(np.array(d.get('T_nopavg')))
n_x = np.array(d.get('n_x'))
n_y = np.array(d.get('n_y'))
n_z = np.array(d.get('n_z'))
print("data dims: " + str(nx) + ' x ' + str(ny) + ' x ' + str(nz) )
print("field dims: " + str(nf))
print("size expected: " + str(nx*ny*nz*nf))

data dims: 59 x 59 x 676
field dims: 51
size expected: 120010956


In [5]:
E2 = np.array(d.get('E2'))
print("E2 - E field profile")
print("size: " + str(np.size(E2)))
print("shape: " + str(np.shape(E2)))

n = np.array(d.get('n'))
print("n - index registry")
print("size: " + str(np.size(n)))
print("shape: " + str(np.shape(n)))

n_oxide = np.array(d.get('n_oxide'))
print("n_oxide - Cu2O index ref")
print("size: " + str(np.size(n_oxide)))
print("shape: " + str(np.shape(n_oxide)))

E2 - E field profile
size: 120010956
shape: (51, 676, 59, 59)
n - index registry
size: 120010956
shape: (51, 676, 59, 59)
n_oxide - Cu2O index ref
size: 51
shape: (1, 51)


In [6]:
if (0):
    omega = 2*math.pi*f
    eps0 = 8.85e-34
    #w = widgets.IntProgress(value = 0,min = 0,max=nx,description="Computing E2: ")
    #display(w)
    Pabs = np.empty((nf,nz,ny,nx))
    Ngen = np.empty((nf,nz,ny,nx))
    Pabs.fill(0.0)
    Ngen.fill(0.0)
    for xx in range(0,nx):
        for yy in range(0,ny):
            for zz in range(0,nz):
                for ff in range(0,nf):
                    if n_oxide[0,ff] == n[ff,zz,yy,xx]:
                        ieps = eps0*(2*n_oxide[0,ff][0]*n_oxide[0,ff][1])
                        o = omega[0,ff]
                        Pabs[ff,zz,yy,xx] = 0.5*o*E2[ff,zz,yy,xx]*ieps
                        Ngen[ff,zz,yy,xx] = Pabs[ff,zz,yy,xx]*1e-6/(6.626e-34*f[0,ff])
        print(str(xx+1) + ' out of ' + str(nx) + ' complete.')

    #d.create_dataset("E2",data=E2)
    #d.flush()
    d.create_dataset("Pabs",data=Pabs)
    d.create_dataset("Ngen",data=Ngen)
    d.flush()
else:
    Pabs = np.array(d.get("Pabs"))
    Ngen = np.array(d.get("Ngen"))


In [7]:
d.close()

In [8]:
#start app instance 
app = QtGui.QApplication([])

#GraphicsView and layout base instances
view = pg.GraphicsView()
view.resize(1200,800)
lay = pg.GraphicsLayout()
view.setCentralItem(lay)
view.setWindowTitle("Data Browser")

#datmat = E2
datmat = Ngen
#datmat = Pabs

yscale = (np.max(y)-np.min(y))/ny
xscale = (np.max(x)-np.min(x))/nx
zscale = (np.max(z)-np.min(z))/nz

# Interpret image data as row-major instead of col-major
pg.setConfigOptions(imageAxisOrder='row-major')

p1 = lay.addPlot(title="R",col=1,row=1,colspan=5)
p1.plot(wl*1e-9,R)
p1_xax = p1.getAxis('bottom')
p1_xax.setLabel(text='Wavelength',units='m')
p1_yax = p1.getAxis('left')
p1_yax.setLabel(text='Fraction')
l1 = pg.InfiniteLine(pos=(np.max(wl)-np.min(wl))/2*1e-9,bounds=[np.min(wl)*1e-9,np.max(wl)*1e-9],angle=90,movable=True)
p1.addItem(l1)

p2 = lay.addPlot(col=1,row=2,colspan=4)
p2.invertY(False)
p2.setAspectLocked()

img2=pg.ImageItem()
img2.setImage(datmat[26,337,:,:])
p2.addItem(img2)
p2_yax = p2.getAxis('left')
p2_yax.setScale(scale=yscale)
p2_yax.setLabel(text='y',units='m')
p2_xax = p2.getAxis('bottom')
p2_xax.setScale(scale=xscale)
p2_xax.setLabel(text='x',units='m')

hist_lut2 = pg.HistogramLUTItem()
hist_lut2.setImageItem(img2)

lay.addItem(hist_lut2,col=5,row=2,colspan=1)

p3 = lay.addPlot(col=6,row=1,colspan=1,rowspan=2)
p3.setAspectLocked()
img3=pg.ImageItem()
img3.setImage(datmat[26,:,:,29])
p3.addItem(img3)
p3_xax = p3.getAxis('bottom')
p3_xax.setScale(scale=xscale)
p3_xax.setLabel(text='x',units='m')
p3_yax = p3.getAxis('left')
p3_yax.setScale(scale=zscale)
p3_yax.setLabel(text='z',units='m')

l3 = pg.InfiniteLine(pos=337,angle=0,movable=True,bounds=[0,nz-1])
p3.addItem(l3)

p4 = lay.addPlot(col=7,row=1,colspan=1,rowspan=2)
p4.setAspectLocked()
img4 = pg.ImageItem(view=pg.PlotItem())
img4.setImage(datmat[26,:,29,:])
p4.addItem(img4)

p4_xax = p4.getAxis('bottom')
p4_xax.setScale(scale=yscale)
p4_xax.setLabel(text='x',units='m')
p4_yax = p4.getAxis('left')
p4_yax.setScale(scale=zscale)
p4_yax.setLabel(text='z',units='m')

l4 = pg.InfiniteLine(pos=337,angle=0,movable=True,bounds=[0,nz-1])
p4.addItem(l4)

p4.setXLink(p3)
p4.setYLink(p3)

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return idx, array[idx]

def updatep1(il):
    #print(il.value())
    wl_ind, wl_near = find_nearest(wl*1e-9,il.value())
    il.setValue(wl_near)
    p1.setTitle('R, wl = ' + str(round(wl_near*1e9,1))+'nm')
    img2.setImage(datmat[wl_ind,round(l3.value()),:,:])
    img3.setImage(datmat[wl_ind,:,:,29])
    img4.setImage(datmat[wl_ind,:,29,:])
    hist_lut2.autoHistogramRange()

def updatep3(il):
    wl_ind, wl_near = find_nearest(wl*1e-9,l1.value())
    px = round(il.value())
    l4.setValue(il.value())
    img2.setImage(datmat[wl_ind,px,:,:])
    hist_lut2.autoHistogramRange()
    
def updatep4(il):
    wl_ind, wl_near = find_nearest(wl*1e-9,l1.value())
    px = round(il.value())
    l3.setValue(il.value())
    img2.setImage(datmat[wl_ind,px,:,:])
    hist_lut2.autoHistogramRange()

l1.sigPositionChanged.connect(updatep1)
updatep1(l1)

l3.sigPositionChanged.connect(updatep3)
updatep3(l3)

l4.sigPositionChanged.connect(updatep4)
updatep4(l4)

view.show()

## Start Qt event loop unless running in interactive mode or using pyside.
if __name__ == '__main__':
    import sys
    if (sys.flags.interactive != 1) or not hasattr(QtCore, 'PYQT_VERSION'):
        QtGui.QApplication.instance().exec_()
del app,view,lay